# Recommender systems

## One of the most common uses of big data is to predict and suggest what users may want.  This allows Google to show you relevant ads or to suggest news in Google Now; Amazon to recommend relevant products; Netflix to recommend movies that you might like; or most recently, the famous **Weekly Dicovery** of Spotify.

## All these products are based on systems of recommendation: a information retrieval method to provide users with relevant, yet novel and diverse, information. 

## In this class we will use a pretty famous dataset based on movies ratings, 'MovieLens', to learn the basics of recommender systems. 

## Table of Contents (times are approximated)

1. [Getting and analysing some data (1.5-2 h, typically until break)](#data)
2. [Most popular movies (0.5-1 min)](#popular)
3. [Metrics for recommender systems (2-2.5h)](#metrics)

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import io
import os

<a id='data'></a>
## 1.1 Load data

We will use MovieLens dataset, which is one of the most common datasets used when implementing and testing recommender engines. This data set consists of:
* 100,000 ratings (1-5) from 943 users on 1682 movies. 
* Each user has rated at least 20 movies. 
* Simple demographic info for the users (age, gender, occupation, zip)

The data was collected through the MovieLens [website](https://movielens.org) during the seven-month period from September 19th, 
1997 through April 22nd, 1998. This data has been cleaned up - users
who had less than 20 ratings or did not have complete demographic
information were removed from this data set.

You can download the dataset [here](http://files.grouplens.org/datasets/movielens/ml-100k.zip).

Take a look at the readme file!!!

In [10]:
data_root = "ml-100k/"

In [11]:
!cat {data_root}README

SUMMARY & USAGE LICENSE

MovieLens data sets were collected by the GroupLens Research Project
at the University of Minnesota.
 
This data set consists of:
	* 100,000 ratings (1-5) from 943 users on 1682 movies. 
	* Each user has rated at least 20 movies. 
        * Simple demographic info for the users (age, gender, occupation, zip)

The data was collected through the MovieLens web site
(movielens.umn.edu) during the seven-month period from September 19th, 
1997 through April 22nd, 1998. This data has been cleaned up - users
who had less than 20 ratings or did not have complete demographic
information were removed from this data set. Detailed descriptions of
the data file can be found at the end of this file.

Neither the University of Minnesota nor any of the researchers
involved can guarantee the correctness of the data, its suitability
for any particular purpose, or the validity of results based on the
use of the data set.  The data set may be used for any research
purposes under th

In [12]:
columns = ['user_id','item_id','rating','timestamp']
datafile = os.path.join(data_root, "u.data")
data = pd.read_csv(datafile, sep='\t', names=columns)
data.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


### A remainder of the numpy library

*Pandas library is nothing alse than numpy under the hood (numpy with steroids, if you like). You can access the data (in matrix from) with he "values" attribute, e.g. data.values*

In [14]:
# The attribute shape provides the shape of the matrix
data.shape

(100000, 4)

In [21]:
# Note that if we return the first column, we get a vector (of 100000 components)
data.values[:,0]

array([196, 186,  22, ..., 276,  13,  12])

In [22]:
# same with the first row (this time, we get a vector of 4 components)
data.values[0]

array([      196,       242,         3, 881250949])

In [23]:
# access all rows, and first 3 columns 
data.iloc[:,:3]

,user_id,item_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1
...,...,...,...
99995,880,476,3
99996,716,204,5
99997,276,1090,1
99998,13,225,2


In [24]:
# access all collumns, and first 3 rows 
data.iloc[:3,:]

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116


In [28]:
# access first 10 rows
data.iloc[:10]
# This is equivalent to data.values[:10]

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
5,298,474,4,884182806
6,115,265,2,881171488
7,253,465,5,891628467
8,305,451,3,886324817
9,6,86,3,883603013


In [29]:
# access first column
data.iloc[:,0]

0        196
1        186
2         22
3        244
4        166
        ... 
99995    880
99996    716
99997    276
99998     13
99999     12
Name: user_id, Length: 100000, dtype: int64

In [30]:
# Number of users and items
n_users = data.user_id.nunique()

In [31]:
# Number of items
n_items = data.item_id.nunique()

In [32]:
print("There are %s users and %s items" %(n_users, n_items))

There are 943 users and 1682 items


## 1.2 A dictionary for movies and a search tool

In order to analyze the predicted recommendations, let's create a python dictonary that will allow us to translate any item id to the corresponding movie title. Also, let's write a small function that returns the ids of the movies containing some text.

The correspondance between titles and ids is stored in the u.item file

In [33]:
data_root = "ml-100k/"
items_id_file = os.path.join(data_root, "u.item")
!head $items_id_file

1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0
2|GoldenEye (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?GoldenEye%20(1995)|0|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0
3|Four Rooms (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995)|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0
4|Get Shorty (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Get%20Shorty%20(1995)|0|1|0|0|0|1|0|0|1|0|0|0|0|0|0|0|0|0|0
5|Copycat (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Copycat%20(1995)|0|0|0|0|0|0|1|0|1|0|0|0|0|0|0|0|1|0|0
6|Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)|01-Jan-1995||http://us.imdb.com/Title?Yao+a+yao+yao+dao+waipo+qiao+(1995)|0|0|0|0|0|0|0|0|1|0|0|0|0|0|0|0|0|0|0
7|Twelve Monkeys (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Twelve%20Monkeys%20(1995)|0|0|0|0|0|0|0|0|1|0|0|0|0|0|0|1|0|0|0
8|Babe (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Babe%20(1995)|0|0|0|0|1

### *Simple reminder of dictionaries*

In [34]:
aux = {'hola': 'que haces?', 1: '237'}

In [35]:
# Access value of key='hola'
aux['hola']

'que haces?'

In [36]:
# create new key
aux['nueva']='nuevisimo'

In [37]:
# Update value of existing key
aux['nueva']='menos nuevo'

In [48]:
# get all keys 
aux.keys()

dict_keys(['hola', 1, 'nueva'])

In [49]:
# get all values
aux.values()

dict_values(['que haces?', '237', 'menos nuevo'])

In [50]:
# get all keys and values at the same time
aux.items()

dict_items([('hola', 'que haces?'), (1, '237'), ('nueva', 'menos nuevo')])

In [59]:
# Create a dictionary for movie titles and ids
item_dict = {}
with io.open(items_id_file, 'rb') as f:
    for line in f.readlines():
        record = line.split(b'|')
        item_dict[int(record[0])] = str(record[1])
    
# We can use this dict to see the films a user has seen, for instance. 
for record in data.values[:20]:
    print("User {u} viewed '{m}' and gave a {r} rating".format(
        u=record[0], m=item_dict[record[1]], r=record[2]))    

User 196 viewed 'b'Kolya (1996)'' and gave a 3 rating
User 186 viewed 'b'L.A. Confidential (1997)'' and gave a 3 rating
User 22 viewed 'b'Heavyweights (1994)'' and gave a 1 rating
User 244 viewed 'b'Legends of the Fall (1994)'' and gave a 2 rating
User 166 viewed 'b'Jackie Brown (1997)'' and gave a 1 rating
User 298 viewed 'b'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)'' and gave a 4 rating
User 115 viewed 'b'Hunt for Red October, The (1990)'' and gave a 2 rating
User 253 viewed 'b'Jungle Book, The (1994)'' and gave a 5 rating
User 305 viewed 'b'Grease (1978)'' and gave a 3 rating
User 6 viewed 'b'Remains of the Day, The (1993)'' and gave a 3 rating
User 62 viewed 'b'Men in Black (1997)'' and gave a 2 rating
User 286 viewed 'b"Romy and Michele's High School Reunion (1997)"' and gave a 5 rating
User 200 viewed 'b'Star Trek: First Contact (1996)'' and gave a 5 rating
User 210 viewed 'b'To Wong Foo, Thanks for Everything! Julie Newmar (1995)'' and gave a 3 

In [62]:
# Define a function that retrieves all the ids and titles for movies containing 'text' in its title
def returnItemId(text, ids):
    """
    :param text: string to be looked for in movies titles
    :param ids: dicttionary of {id:title}
    
    :return: a list of (id,title) if text found in titles, and an empty list otherwise.
    """
    # convert input text to lowercase
    text_ = text.lower()
    # find occurances
    search = []
    for id_,title in ids.items():
        if text_ in title.lower():
            search.append((id_,title))
    
    return search

In [63]:
returnItemId('but', item_dict)

[(240, "b'Beavis and Butt-head Do America (1996)'"),
 (435, "b'Butch Cassidy and the Sundance Kid (1969)'"),
 (580,
  "b'Englishman Who Went Up a Hill, But Came Down a Mountain, The (1995)'"),
 (1401, "b'M. Butterfly (1993)'"),
 (1459, "b'Madame Butterfly (1995)'"),
 (1614, "b'Reluctant Debutante, The (1958)'"),
 (1621, "b'Butterfly Kiss (1995)'"),
 (1645, "b'Butcher Boy, The (1998)'"),
 (1650, "b'Butcher Boy, The (1998)'")]

## 1.3 Data consistency (always double check everything!!!)

In [75]:
# check whether titles are unique or not. They are not!!!
np.unique(list(item_dict.values())).size

1664

In [76]:
np.unique(list(item_dict.keys())).size

1682

### One work around: create another dict that consolidates ids with the same movie title

Alternatively, you could create a Dataframe from the above dictionary of ids:titles

<pre><code>
dd = pd.DataFrame.from_dict(item_dict, orient='index').reset_index()
dd.columns = ['item_id', 'title']
dd.head()
</code></pre>

find the duplicates (`dd.groupby('title')...`), and create a new id for the items. 

In [ ]:
duplicates_item_dict = {}
# Las claves en "duplicates_item_dict" son los nombres de las películas
# Los valores son una lista de los ids (que pueden ser uno solo, o varios)
for id_, name in list(item_dict.items()):
    # clave: name; valor: list(id_)
    ?

# show the duplicated titles
for k,v in list(duplicates_item_dict.items()):
    if len(v)>1:
        print(k,v)

Create a dict where the key are the original ids, and the values are the unique one. 
We will use this dictionary to remove duplicates in a dataframe.

In [ ]:
unique_id_item_dict ={}
for new_id, old_id_list in enumerate(duplicates_item_dict.values()):
    # key: old_id; value: new_id
    ?

unique_id_item_dict = {old_id: new_id for new_id, old_id_list in enumerate(duplicates_item_dict.values()) 
                                      for old_id in old_id_list}

In [ ]:
unique_id_item_dict

Create another dict mapping moving titles to this new unique id

In [ ]:
unique_item_dict = {unique_id_item_dict[k]:v 
                    for k,v in item_dict.items()}
    
assert(len(set(unique_item_dict.keys())) == 
       len(set(unique_item_dict.values())))

Now we can use our `returnItemId()` mehtod safely =)

In [ ]:
returnItemId('but', unique_item_dict)

In [ ]:
returnItemId('but', item_dict)

## 1.4 Train and test sets

GroupLens provides several splits of the dataset, so that we can check the goodness of our algorithms. See the README file for more  details. Here we will use one of such splits.

Please notice that we have to correct for the non-unique movie's id issue!!

In [77]:
!ls $data_root

README	   u.data   u.item	  u1.base  u2.test  u4.base  u5.test  ub.base
allbut.pl  u.genre  u.occupation  u1.test  u3.base  u4.test  ua.base  ub.test
mku.sh	   u.info   u.user	  u2.base  u3.test  u5.base  ua.test


In [78]:
trainfile = os.path.join(data_root, 'ua.base')
!head $trainfile

1	1	5	874965758
1	2	3	876893171
1	3	4	878542960
1	4	3	876893119
1	5	3	889751712
1	6	5	887431973
1	7	4	875071561
1	8	1	875072484
1	9	5	878543541
1	10	3	875693118


In [79]:
columns = ['user_id', 'item_id', 'rating', 'timestamp']
trainfile = os.path.join(data_root, "ua.base")
train = pd.read_csv(trainfile,sep='\t',names=columns)
print('There are %s users, %s itmes and %s pairs in the train set' \
      %(train.user_id.unique().shape[0], train.item_id.unique().shape[0], train.item_id.count()))
train.head()


There are 943 users, 1680 itmes and 90570 pairs in the train set


,user_id,item_id,rating,timestamp
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712


In [80]:
# same for test
columns = ['user_id', 'item_id', 'rating', 'timestamp']
testfile = os.path.join(data_root, "ua.test")
test = pd.read_csv(testfile,sep='\t',names=columns)
print('There are %s users, %s itmes and %s pairs in the test set' \
      %(test.user_id.unique().shape[0], test.item_id.unique().shape[0], test.item_id.count()))
test.head()


There are 943 users, 1129 itmes and 9430 pairs in the test set


,user_id,item_id,rating,timestamp
0,1,20,4,887431883
1,1,33,4,878542699
2,1,61,4,878542420
3,1,117,3,874965739
4,1,155,2,878542201


### Correcting for non-unique movies id 

Use Pandas `apply` method. Note it can be applied to all rows in a column, thus returning a Pandas Series; or it can be applied row-wise, he

*Reminder of lambda functions in Python: is a way of calling short functions (1 line of code), without having to define the function in a separted cell.*

In [ ]:
train['item_id'] = train.item_id.apply(lambda id: my_dict(id))

In [ ]:
test['item_id'] = ?

<a id='popular'></a>
## 2. Most popular movies

Recommending popular items is a simple, yet quite effective baseline for recommendation. Indeed, most RS suffer from a strong *popularity bias*, i.e. they tend to recommend popular items more frequently than they should -just because suggesting what is popular is effective!-. There is a lot of research  devote to understand this behaviour and to develop recipies to avoid it. 

Movies can be ranked according to different popularity metrics:
* Most rated movie (it is assumed that this is the most watched movie)
* Most positively rated movie (rating > 4.0)
* Highest rated movie

## 2.1 Most rated movie

In [81]:
# group the train dataset by item and count the number of users using Pandas
mostRated = train.groupby('item_id')['user_id'].count()
mostRated.head()

item_id
1    392
2    121
3     85
4    198
5     79
Name: user_id, dtype: int64

In [82]:
mostRated.head()

item_id
1    392
2    121
3     85
4    198
5     79
Name: user_id, dtype: int64

In [83]:
# sort in descending order
mostRatedSorted = mostRated.sort_values(ascending=False)

In [84]:
mostRatedSorted.head()

item_id
50     495
100    443
181    439
258    412
286    400
Name: user_id, dtype: int64

In [234]:
# add the movie title, using the apply method, returning a Pandas Series in each row
def add_title(row):
    id_ = row[0]
    title_ = item_dict[id_]
    freq_ = row[1]
    new_row = [id_,title_,freq_]
    return pd.Series(new_row, index=['id', 'title', 'freq'])

mostRatedMovies = (mostRatedSorted
                   .reset_index()
                   .apply(add_tile, axis=1)
                  )

In [235]:
mostRatedMovies

,id,title,freq
0,50,b'Star Wars (1977)',495
1,100,b'Fargo (1996)',443
2,181,b'Return of the Jedi (1983)',439
3,258,b'Contact (1997)',412
4,286,"b'English Patient, The (1996)'",400
...,...,...,...
1675,1568,b'Vermont Is For Lovers (1992)',1
1676,1569,"b'Vie est belle, La (Life is Rosey) (1987)'",1
1677,1570,b'Quartier Mozart (1992)',1
1678,1571,b'Touki Bouki (Journey of the Hyena) (1973)',1


## 2.2 Most positively rated movie

Use the query method to filter values in train, `train.query('rating>4')`, or the traditional form `train[train.rating>4]`.

In [165]:
mostPositiveRated=train[train['rating']>4].groupby('item_id')['user_id'].count()
mostPositiveSorted = mostPositiveRated.sort_values(ascending=False)
mostPositiveRatedMovies = (mostPositiveSorted
                   .reset_index()
                   .apply(add_title, axis=1)
                  )
mostPositiveRatedMovies.head(20)

#Se observa que hay un sesgo en cuanto a películas 'frikis'

,id,title,freq
0,50,b'Star Wars (1977)',277
1,100,b'Fargo (1996)',194
2,174,b'Raiders of the Lost Ark (1981)',181
3,127,"b'Godfather, The (1972)'",179
4,56,b'Pulp Fiction (1994)',170
5,318,"b""Schindler's List (1993)""",168
6,98,"b'Silence of the Lambs, The (1991)'",167
7,172,"b'Empire Strikes Back, The (1980)'",159
8,64,"b'Shawshank Redemption, The (1994)'",152
9,181,b'Return of the Jedi (1983)',146


## 2.3 Highest mean rating movie

In [273]:
# obtain the highest rated movies, with a minium number of users/ratings.
min_ratings = 50
mean_rating = train.groupby('item_id').agg({'rating':np.mean,'user_id':'count'})
mejores_peliculas=mean_rating[mean_rating['user_id']>min_ratings].sort_values('rating',ascending=False).reset_index()
item_dict[mejores_peliculas.loc[0,'item_id']]

"b'Wallace & Gromit: The Best of Aardman Animation (1996)'"

In [274]:
# Otra forma
meanRated=train.groupby('item_id')['rating'].apply(list)
meanRated=meanRated.reset_index()
meanRated=meanRated[meanRated['rating'].apply(lambda lista: len(lista)>min_ratings)]
meanRated['rating']=meanRated['rating'].apply(np.mean)
meanSorted = meanRated.sort_values('rating',ascending=False)
meanRatedMovies = (meanSorted
                   .apply(add_title, axis=1)
                  )
meanRatedMovies.head(20)

,id,title,freq
113,114.0,b'Wallace & Gromit: The Best of Aardman Animat...,4.491525
407,408.0,"b'Close Shave, A (1995)'",4.480769
168,169.0,"b'Wrong Trousers, The (1993)'",4.476636
317,318.0,"b""Schindler's List (1993)""",4.475836
482,483.0,b'Casablanca (1942)',4.459821
63,64.0,"b'Shawshank Redemption, The (1994)'",4.457364
11,12.0,"b'Usual Suspects, The (1995)'",4.386454
602,603.0,b'Rear Window (1954)',4.374359
49,50.0,b'Star Wars (1977)',4.365657
177,178.0,b'12 Angry Men (1957)',4.327434


<div class  = "alert alert-info"> 
** QUESTION **: set the value of *min_ratings* to 1, and re-run the cell. What happens now? Change this value
</div>

<div class  = "alert alert-info"> 
** QUESTION **: Which method is better?? How to measure a recommender system? 
</div>

<div class  = "alert alert-info"> 
** IMPORTANT QUESTION **: When might be useful to recommend popular items?
</div>

<a id='metrics'></a>
## 3. Metrics for recommender systems

As we have seen, even with the simplest solution --aka, recommending popular items-- is difficult to known which technique performs better. For this, there are a number of metrics that allow one to measure the goodness of a recommender system. 

Metrics can be design for measuring the relevance or accuracy of a recommendation, but they can be created for evaluating the novelty of a recommendation, or its diversity. 

For now, we will focus on relevance and accuracy. Several metrics exist:
* Accuracy: rmse, mae.
* Not ranked: Recall@k, Precision@k.
* With rank disccount: map@k, ndcg@k.
* With rank ordering: mean percentile rank.

We will be definiing some of them whitin this class. For the moment, let's talk about precision and recall.

## 3.1 Precision and recall

<img src="https://upload.wikimedia.org/wikipedia/commons/2/26/Precisionrecall.svg" alt="Precision and Recall in IR" style="float: right; width: 300px"/>

The concept of precision and recall comes form the world of information retrieval, have a look at the wikipedia:

https://en.wikipedia.org/wiki/Precision_and_recall

From this entry:

 * "**precision** (also called positive predictive value) is the fraction of retrieved instances that are relevant".
 * "**recall** (also known as sensitivity) is the fraction of relevant instances that are retrieved".

<br />
<div class  = "alert alert-info"> 
** QUESTION **: how do we know if some movie, unknown to the user, is relevant?
</div>

In other words, we cannot measure a false positive --something recommended that was not relevant--. In this regard, only recall-oriented metrics have an actual meaning in RS. Nonetheless, its common practice to define both metrics in RS as follows:
 
### $$\mathrm{recall}@N = \frac{\sum_{k=1}^N rel(k)}{\sum_{i\in \mathcal{I}_u} 1}$$
### $$\mathrm{precision}@N = \frac{\sum_{k=1}^N rel(k)}{N}$$

Here, $\mathcal{I}_u$ is the set of items adopted by user $u$, and $rel(k)$ is the relevance of a recommendation at position k in the list of recommendations. For ratings, the relevance could be defined as those movies rated above a certain threshold, e.g. $r_{ui}>4.0$. 

**Important to note: since precision is pretty much the same as recall in RS, metrcis usch as the *area under the ROC curve* doesn't have any meaning!!**

<div class = "alert alert-success">
As an example, consider a user that watched the following films:
<br /><br />
'Designated Mourner, The (1997)'
<br />
'Money Talks (1997)'
<br />
'Madame Butterfly (1995)'
<br />
'Batman Forever (1995)'
<br /><br />
The recommended items were: 
<br /><br />
'Batman (1989)' 
<br />
'Madame Butterfly (1995)'
<br /><br />
**What would be the recall and precision @1? and @2?**
<br />
**What do you think of recommending Batman? Is a bad or a good recommendation?**
</div>

Please notice that there isn't any actual difference between precision and recall in the context of RS: both measure the relevance of the recommendations, and tell nothing about items recommended that haven't been adopted by the user. Thus, it make sense to define a normalized recall as:

### $$\mathrm{recall}@N = \frac{\sum_{i=1}^N rel_i}{\mathrm{min}(N, \sum_{i\in \mathcal{I}_u} 1})$$

This way, results are normalized to 1 always.

<div class="alert alert-success">
**Exercise** Implement the above definition of recall
</div>

In [195]:
def recall_at_n(N, test, recommended, train=None):
    """
    :param N: number of recommendations
    :param test: list of movies seen by user in test
    :param train: list of movies seen by user in train. This has to be removed from the recommended list 
    :param recommended: list of movies recommended
    
    :return the recall
    """
    if train is not None: # Remove items in train
        rec_true = []
        for r in recommended:
            ?
    else:
        rec_true = recommended    
    intersection = len(set(recommended[:N]) & set(test))
    return intersection / float(np.minimum(N, len(test)))

In [202]:
seen = ['Designated Mourner, The (1997)', 'Money Talks (1997)', 'Madame Butterfly (1995)', 'Batman Forever (1995)']
recommended = ['Batman (1989)', 'Madame Butterfly (1995)']

In [203]:
recall_at_n(1, seen, recommended)

0.0

In [204]:
recall_at_n(2, seen, recommended)

0.5

In [205]:
# Check it's well normalized
print(recall_at_n(3, seen, recommended))
print(recall_at_n(10, seen, recommended))
print(recall_at_n(100, seen, recommended))

0.3333333333333333
0.25
0.25


### Now, use this implementation to measure the efficiency of the popularity baselines in the test set. Use the top-5 movies, for instance

In [275]:
mostRatedMovies.iloc[:5,:]
recommended=mostRatedMovies.iloc[:10,0]

In [276]:
mostPositiveRatedMovies.iloc[:5,1:]

,title,freq
0,b'Star Wars (1977)',277
1,b'Fargo (1996)',194
2,b'Raiders of the Lost Ark (1981)',181
3,"b'Godfather, The (1972)'",179
4,b'Pulp Fiction (1994)',170


In [283]:
meanRatedMovies.iloc[:5,1:]
recommended=meanRatedMovies.iloc[:,0]

In [278]:
train.head()

,user_id,item_id,rating,timestamp
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712


*Since `recall_at_n` takes both train and test list per user, we need to create a dataset with the list of movies seen in train and test*

Thus, get the list of movies per user in train and test, and join the two dataframes. For the join, use the pandas method `merge`.

In [279]:
# get movies in train per user. For this, group by user and get a list of item ids.
trainUsersGrouped = train.groupby('user_id')['item_id'].apply(list).reset_index()

In [280]:
# same with test data
testUsersGrouped = test.groupby('user_id')['item_id'].apply(list).reset_index()
testUsersGrouped

,user_id,item_id
0,1,"[20, 33, 61, 117, 155, 160, 171, 189, 202, 265]"
1,2,"[13, 50, 251, 280, 281, 290, 292, 297, 312, 314]"
2,3,"[245, 294, 323, 328, 331, 332, 334, 335, 337, ..."
3,4,"[50, 260, 264, 288, 294, 303, 354, 356, 357, 361]"
4,5,"[1, 2, 17, 98, 110, 225, 363, 424, 439, 454]"
...,...,...
938,939,"[106, 121, 258, 409, 476, 689, 931, 993, 1054,..."
939,940,"[14, 66, 193, 205, 272, 289, 315, 568, 655, 873]"
940,941,"[7, 15, 117, 124, 147, 181, 257, 258, 475, 993]"
941,942,"[117, 200, 261, 323, 423, 427, 487, 584, 604, ..."


In [288]:
trainUsersGrouped.item_id.apply(lambda seen: recall_at_n(10,seen, recommended)).mean()

0.22004241781548178

In [289]:
testUsersGrouped.item_id.apply(lambda seen: recall_at_n(10,seen, recommended)).mean()

0.02439024390243906

In [ ]:
# make the join: use pandas merge method
joined = ?

In [ ]:
joined.head()

In [ ]:
joined.item_id_test.head()

In [ ]:
# How would you access values in test?
?

In [ ]:
# This second method is easier if we want to access several columns at once, and operate over them.
# For instance, if we like to concatenate both train and test list, we will do:
?

In [ ]:
# Use the above method to calculate the recall of the mostRatedMovies recommendation, for each user:
?

*As you can see, some users have a quite large recall (0.5), while for others is small (e.g, 0.14). Let's calculate the mean.*

In [ ]:
topN = 30
# calculate the average recall across all users for mostRatedMovies recommendation
recall_per_user = ?
recall_per_user.mean()

In [ ]:
# calculate the average recall across all users for positiveRatedMovies recommendation
?

In [ ]:
# calculate the average recall across all users for meanRatedMovies recommendation
?

## 3.2 Mean Averaged Precision (MAP) -- Advanced material

Previous metrics did not account for the ranking of the recommendation, i.e. the relative position of a movie within the sorted list of recommendations. **But orders matters!** Metrics like MAP, MRR or NDCG try to tackle down this problem. 

From the blog *http://fastml.com/what-you-wanted-to-know-about-mean-average-precision/*:

> Here’s another way to understand average precision. Wikipedia says AP is used to score document retrieval. You can think of it this way: you type something in Google and it shows you 10 results. It’s probably best if all of them were relevant. If only some are relevant, say five of them, then it’s much better if the relevant ones are shown first. It would be bad if first five were irrelevant and good ones only started from sixth, wouldn’t it? AP score reflects this.

Implementation taken from:

https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py



## Average Precision 

The Average Precision is definied as:

### $$\mathrm{AP}@N = \frac{\sum_{k=1}^N P(k) \times rel(k)}{\mathrm{min}(N, \sum_{i\in \mathcal{I}_u} 1)}$$

where $P(k)$ is the precision at cut-off in the item list, i.e. the ratio of the number of recommended items adopted, up to the position k, over the number k. Thus:

### $$\mathrm{AP}@N = \frac{\sum_{k=1}^N \left(\sum_{i=1}^k rel(i)\right)/k \times rel(k)}{\mathrm{min}(N, \sum_{i\in \mathcal{I}_u} 1)}$$



<div class = "alert alert-success">
Following the example above, consider a user that watched the following films:
<br /><br />
'Designated Mourner, The (1997)'
<br />
'Money Talks (1997)'
<br />
'Madame Butterfly (1995)'
<br />
'Batman Forever (1995)'
<br /><br />
The recommended items were: 
<br /><br />
'Batman (1989)' 
<br />
'Madame Butterfly (1995)'
<br /><br />

<div class = "alert alert-success">
**Calculate AP@1**
<br /><br />
First, *rel(1)=0*, because Batman was not viewed. Also, *P(1) = 0*. Thus, AP@1=0.
<br />
**Calculate AP@2**
<br /><br />
As before, *rel(1)=0*, so the first term does not contribute. For the second term, *rel(2)=1*, so that *P(2)=0.5*. The numerator is hence:
<br /><br />
$P(1)*rel(1)+P(2)*rel(2)=0*0+0.5*1$
<br /><br />
For the denominator, $N=2$ and $\sum_{i\in \mathcal{I}_u} 1)=4$, thus:
<br /><br />
AP@2 = 0.5/2 = 0.25
</div>

Let's now implement it =)

In [ ]:
def apk(N, test, recommended, train=None):
    """
    Computes the average precision at N given recommendations.
    
    :param N: number of recommendations
    :param test: list of movies seen by user in test
    :param train: list of movies seen by user in train. This has to be removed from the recommended list 
    :param recommended: list of movies recommended
    
    :return The average precision at N over the test set
    """
    if train is not None: 
        rec_true = []
        for r in recommended:
            if r not in train:
                rec_true.append(r)
    else:
        rec_true = recommended    
    predicted = rec_true[:N] # top-k predictions
    
    score = 0.0 # This will store the numerator
    num_hits = 0.0 # This will store the sum of rel(i)

    for i,p in enumerate(predicted):
        if p in test and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits/(i+1.0)

    return score / min(len(test), N)

In [ ]:
seen = ['Designated Mourner, The (1997)', 'Money Talks (1997)', 'Madame Butterfly (1995)', 'Batman Forever (1995)']
recommended = ['Madame Butterfly (1995)', 'Batman (1989)']

In [ ]:
apk(1, seen, recommended)

In [ ]:
apk(2, seen, recommended)

In [ ]:
apk(3, seen, recommended)

## MAP

Mean avergae precision is nothing else than the AP averaged across users ;)

Apply it to popularity baselines

In [ ]:
?

<div class="alert alert-success">
The rest of the class is covered in a different notebook
</div>